In [ ]:
import pandas as pd
from pathlib import Path
from io import StringIO
import json

# col cuore in gola

dataset

In [ ]:
data_text = ""

for filename in sorted(["ds", "ts_hs", "ts_ht"]):
    with open(Path("downloads", "task_0", f"{filename}.tsv"), "rt", encoding="utf8") as f:
        data_text += f.read()

df = pd.read_csv(StringIO(data_text), sep="\t")
# dup_idx = df.index.difference(df.drop_duplicates().index)
# df.loc[dup_idx]
df = df.drop_duplicates().reset_index(names="old_idx").reset_index(names="new_idx")

train test indexes

In [ ]:
lookup = df.set_index("old_idx")["new_idx"]
pth = Path("downloads", "task_0", "train_test_splitting.json")
idx = json.load(open(pth, "rt"))
idx = {k: [lookup[i] for i in lst if i in lookup] for k, lst in idx.items()}


In [ ]:
x, y = df["pp_text"], df["label"]
x_train, x_test = x.loc[idx["train"]], x.loc[idx["test"]]
y_train, y_test = y.loc[idx["train"]], y.loc[idx["test"]]

spans